<a href="https://colab.research.google.com/github/JacopoMangiavacchi/Swift-TensorFlow-Sample-Notebooks/blob/master/LinearRegression_MultiVariable_Swift_TensorFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import Python

func getArrayDataset() -> ([[Float]], [Float]) {
    let np = Python.import("numpy")
    let pd = Python.import("pandas")
    let io = Python.import("io")
    let requests = Python.import("requests")

    let url="https://raw.githubusercontent.com/Dataweekends/zero_to_deep_learning_video/master/data/weight-height.csv"
    let s = requests.get(url).content
    let df = pd.read_csv(io.StringIO(s.decode("utf-8")))

    let dummies = pd.get_dummies(df[["Gender"]])
    let trasnformed = pd.concat([df[["Height", "Weight"]], dummies], 1)
    // print(trasnformed)

    let X1 = trasnformed[["Height"]].values
    let X2 = trasnformed[["Gender_Female"]].values
    let X3 = trasnformed[["Gender_Male"]].values
    let Y = trasnformed[["Weight"]].values
  
    let x1Array = X1.tolist().flatMap{ $0.map{ Float($0)! }}
    let x2Array = X2.tolist().flatMap{ $0.map{ Float($0)! }}
    let x3Array = X3.tolist().flatMap{ $0.map{ Float($0)! }}
    let yArray = Y.tolist().flatMap{ $0.map{ Float($0)! }}
  
    return ([x1Array, x2Array, x3Array], yArray)
}

In [0]:
func minMaxNormalized<T: BinaryFloatingPoint>(_ vector : [T]) -> [T] {
    guard let (min, max) = minimumMaximum(vector) else { return vector }
    return vector.map{ ($0 - min) / (max - min) }
}

func minimumMaximum<T: Comparable>(_ vector: [T]) -> (minimum: T, maximum: T)? {
    guard var minimum = vector.first else { return nil }
    var maximum = minimum

    let start = vector.count % 2
    for i in stride(from: start, to: vector.count, by: 2) {
        let (first, second) = (vector[i], vector[i+1])

        if first > second {
            if first > maximum {
                maximum = first
            }
            if second < minimum {
                minimum = second
            }
        } else {
            if second > maximum {
                maximum = second
            }
            if first < minimum {
                minimum = first
            }
        }
    }

    return (minimum, maximum)
}

In [0]:
let (xArray, yVector) = getArrayDataset()

let yNormalizedVector = minMaxNormalized(yVector)
let xNormalizedArray = [minMaxNormalized(xArray[0]), xArray[1], xArray[2]]

let count = xNormalizedArray[0].count * 3
var xNormalizedVector = [Float](repeating: 0, count: count)

for i in stride(from: 0, to: count, by: 3) {
    xNormalizedVector[i] = xNormalizedArray[0][i/3]
    xNormalizedVector[i+1] = xNormalizedArray[1][i/3]
    xNormalizedVector[i+2] = xNormalizedArray[2][i/3]
}

In [0]:
import TensorFlow

In [0]:
@differentiable()
func linear(input: Tensor<Float>) -> Tensor<Float> {
    return input
}

struct LinearRegression: Layer {
    var l1: Dense<Float>
    init(variables: Int = 1) {
        l1 = Dense<Float>(inputSize: variables, outputSize: 1, activation: linear)
    }
    
    @differentiable(wrt: (self, input))
    func applied(to input: Tensor<Float>) -> Tensor<Float> {
        return l1.applied(to: input)
    }
}

In [6]:
let x = Tensor<Float>(shape: [Int32(10000), 3], scalars: xNormalizedVector)
let y = Tensor<Float>(shape: [Int32(10000), 1], scalars: yNormalizedVector)

print(x.shape)
print(y.shape)

TensorShape(dimensions: [10000, 3])
TensorShape(dimensions: [10000, 1])


In [7]:
let optimizer = SGD<LinearRegression, Float>(learningRate: 0.03)
var model = LinearRegression(variables: 3)


for epoch in 1...2000 {
    let (cost, 𝛁model) = model.valueWithGradient { m -> Tensor<Float> in
        let ŷ = m.applied(to: x)
        return meanSquaredError(predicted: ŷ, expected: y)
    }
    optimizer.update(&model.allDifferentiableVariables, along: 𝛁model)
  
    if epoch % 100 == 0 {
        print("Epoch: \(epoch) Cost: \(cost)")
    }
}

Epoch: 100 Cost: 0.002407352
Epoch: 200 Cost: 0.0023789632
Epoch: 300 Cost: 0.002378894
Epoch: 400 Cost: 0.002378883
Epoch: 500 Cost: 0.0023788742
Epoch: 600 Cost: 0.0023788663
Epoch: 700 Cost: 0.0023788589
Epoch: 800 Cost: 0.0023788523
Epoch: 900 Cost: 0.0023788468
Epoch: 1000 Cost: 0.0023788419
Epoch: 1100 Cost: 0.0023788377
Epoch: 1200 Cost: 0.0023788342
Epoch: 1300 Cost: 0.00237883
Epoch: 1400 Cost: 0.002378828
Epoch: 1500 Cost: 0.002378825
Epoch: 1600 Cost: 0.0023788228
Epoch: 1700 Cost: 0.002378821
Epoch: 1800 Cost: 0.0023788193
Epoch: 1900 Cost: 0.0023788174
Epoch: 2000 Cost: 0.0023788156


In [8]:
print(model.applied(to: [[0.7, 0, 1]]))  //Height, Female, Male

[[0.6700497]]
